# Do all the initialization of objects that don't change

1. Define the x and y axis
1. Define the markers
   * confirmed cases
   * mortality cases
1. Define the lines to have a spline shape (smoothed)
1. Define an mapping of state name to abbreviation

In [ ]:
# Define the x-axis for the plot
$xaxis = [XPlot.Plotly.Graph+Xaxis]::new()
$xaxis.nticks = 23
# https://github.com/d3/d3-time-format/blob/master/README.md
$xaxis.tickformat = '%m-%d'

# Define the y-axis for the plot
$yaxis = [XPlot.Plotly.Graph+Yaxis]::new()

# Define the marker for a new cases point
$newCasesMarker = [XPlot.Plotly.Graph+Marker]::new()
$newCasesMarker.color = "rgb(255, 0, 0)"
$newCasesMarker.Size = 10

# Define the marker for a confirmed cases point
$confirmedCasesMarker = [XPlot.Plotly.Graph+Marker]::new()
$confirmedCasesMarker.color = "	rgb(100,149,237)"
$confirmedCasesMarker.Size = 10

# Define the marker for a point
$mortalityMarker = [XPlot.Plotly.Graph+Marker]::new()
$mortalityMarker.color = "rgb(0, 0, 0)"
$mortalityMarker.Size =7

# Define the line as a spline
$line = [XPlot.Plotly.Graph+Line]::new()
$line.shape = "spline"

# Define the line as a spline
$newCasesline = [XPlot.Plotly.Graph+Line]::new()
$newCasesline.shape = "spline"
$newCasesline.width = 5

# Define a mapping of state name to abbreviation
$stateData = @'
[{"name":"Alabama","abbreviation":"AL"},{"name":"Alaska","abbreviation":"AK"},{"name":"Arizona","abbreviation":"AZ"},{"name":"Arkansas","abbreviation":"AR"},{"name":"California","abbreviation":"CA"},{"name":"Colorado","abbreviation":"CO"},{"name":"Connecticut","abbreviation":"CT"},{"name":"Delaware","abbreviation":"DE"},{"name":"District Of Columbia","abbreviation":"DC"},{"name":"Florida","abbreviation":"FL"},{"name":"Georgia","abbreviation":"GA"},{"name":"Hawaii","abbreviation":"HI"},{"name":"Idaho","abbreviation":"ID"},{"name":"Illinois","abbreviation":"IL"},{"name":"Indiana","abbreviation":"IN"},{"name":"Iowa","abbreviation":"IA"},{"name":"Kansas","abbreviation":"KS"},{"name":"Kentucky","abbreviation":"KY"},{"name":"Louisiana","abbreviation":"LA"},{"name":"Maine","abbreviation":"ME"},{"name":"Maryland","abbreviation":"MD"},{"name":"Massachusetts","abbreviation":"MA"},{"name":"Michigan","abbreviation":"MI"},{"name":"Minnesota","abbreviation":"MN"},{"name":"Mississippi","abbreviation":"MS"},{"name":"Missouri","abbreviation":"MO"},{"name":"Montana","abbreviation":"MT"},{"name":"Nebraska","abbreviation":"NE"},{"name":"Nevada","abbreviation":"NV"},{"name":"New Hampshire","abbreviation":"NH"},{"name":"New Jersey","abbreviation":"NJ"},{"name":"New Mexico","abbreviation":"NM"},{"name":"New York","abbreviation":"NY"},{"name":"North Carolina","abbreviation":"NC"},{"name":"North Dakota","abbreviation":"ND"},{"name":"Ohio","abbreviation":"OH"},{"name":"Oklahoma","abbreviation":"OK"},{"name":"Oregon","abbreviation":"OR"},{"name":"Pennsylvania","abbreviation":"PA"},{"name":"Rhode Island","abbreviation":"RI"},{"name":"South Carolina","abbreviation":"SC"},{"name":"South Dakota","abbreviation":"SD"},{"name":"Tennessee","abbreviation":"TN"},{"name":"Texas","abbreviation":"TX"},{"name":"Utah","abbreviation":"UT"},{"name":"Vermont","abbreviation":"VT"},{"name":"Virginia","abbreviation":"VA"},{"name":"Washington","abbreviation":"WA"},{"name":"West Virginia","abbreviation":"WV"},{"name":"Wisconsin","abbreviation":"WI"},{"name":"Wyoming","abbreviation":"WY"},{"name":"Grand Princess","abbreviation":"GP-Ship","NoCities":true},{"name":"Diamond Princess","abbreviation":"DP-Ship","NoCities":true}]
'@ | ConvertFrom-Json 
$stateData.Count

# Import the data from John Hopkins GitHub repo

## Terms of Use

Please see [CSSEGISandData/COVID-19](https://github.com/CSSEGISandData/COVID-19) for current terms of use.

Here are the terms of use of the data from the repo as of 3/18/2020:

> This GitHub repo and its contents herein, including all data, mapping, and analysis, copyright 2020 Johns Hopkins University, all rights reserved, is provided to the public strictly for educational and academic research purposes. The Website relies upon publicly available data from multiple sources, that do not always agree. The Johns Hopkins University hereby disclaims any and all representations and warranties with respect to the Website, including accuracy, fitness for use, and merchantability. Reliance on the Website for medical guidance or use of the Website in commerce is strictly prohibited.

## What this step does

1. Get the raw CSV from GitHub.  These are separate CSVs for:
   * Confirmed cases
   * Recovered cases
   * Mortality cases
1. Convert each CSV to objects
1. Filters to only the data from the US
1. Creates one object with a property containing a collection of each type of case


In [ ]:
# I considered switching to https://ourworldindata.org/coronavirus-source-data due to licensing, but the data seems much less accurate.

$confirmedString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv -RetryIntervalSec 15 -MaximumRetryCount 3).Content
$mortalityString = (Invoke-WebRequest -Uri https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv -RetryIntervalSec 15 -MaximumRetryCount 3).Content

$usConfirmedAll = $confirmedString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US' }
$usMortalityAll  = $mortalityString | ConvertFrom-CSV | Where-Object { $_.'Country/Region' -eq 'US' }

$usAllData = [pscustomobject] @{
    Confirmed = $usConfirmedAll
    Mortality = $usMortalityAll
}

Write-Verbose "Done $($usAllData.Confirmed.Count) - $($usAllData.Mortality.Count)" -Verbose

# Create the Summaries

1. Create a function to create a summary.
   * This takes two filters because the data changed on 3/9/2020 and this filters to the correct data to use before and after the change.
   * If you don't pass the filter, it creates a sumarry for the entire US
2. Create a function which creates the filters to use to create a state summary and uses the previous function to create a summary for that state.
3. Creates the summary for the US
4. Loops through the state data and creates summaries for every state.

In [ ]:
# Filter to the United States

Function Get-Summary {
    param(
        [ScriptBlock] $StateFilterScriptBlock = { ($_.'Province/State' -notmatch '.*, \w{2}' -or $_.'Province/State' -match '.*Princess' )},
        [parameter(Mandatory)]
        [Object] $UsAllData
    )
    
    
    if(!$usAllData.Confirmed)
    {
        throw "no confirmed"
    }

    $usConfirmedByState = $usAllData.Confirmed | Where-Object $stateFilterScriptBlock
    $usMortalityByState  = $usAllData.Mortality | Where-Object $stateFilterScriptBlock
    
    Write-Verbose "$($usConfirmedByState.Count) confirmed cases States/Ships" 
    Write-Verbose "$($usMortalityByState.Count) mortality States/Ships"  

    if(!$usConfirmedByState)
    {
        return $null
    }
    # Get unique dates on or after 2/15
    $dates = $usConfirmedByState[0] | Get-member -MemberType NoteProperty  | Where-Object {$_.Name -match '\d+/\d+/\d+' -and ([datetime] $_.Name) -ge ([datetime] '3/10/2020')} | Select-Object -ExpandProperty Name | ForEach-Object {
            $date = [datetime] $_
            [pscustomobject]@{
                Name = $_
                Date = $date
            }
        } | Sort-Object -Property Date
    #$dates[-1]

    $sum = ($usConfirmedByState | Where-Object {$_.'Province/State' -eq 'Washington'})."$($dates[-1].Name)" | measure-object -sum | select-object -Property Count, Sum
    Write-Verbose "$($dates[-1].Name) washington state confirmed ($($sum.count)) record: $($sum.Sum)  " 
    Write-Verbose "of $($dates.count) days" 
    
    # Summarize the data for each date
    $usSummary = @()
    $dateBoundary = [datetime]'3/9/2020'
    foreach($dateObject in $dates)
    {
        $date = $dateObject.Date
        $dateString = $dateObject.Name

        $confirmedSum = ($usConfirmedByState | Measure-Object -Sum -Property $dateString).Sum
        $mortalitySum = ($usMortalityByState | Measure-Object -Sum -Property $dateString).Sum    

        $usSummary += [pscustomobject] @{
            date = [datetime] $date
            confirmed = $confirmedSum
            deaths = $mortalitySum
        }
    }
    $usSummary = $usSummary | Sort-Object -Property Date
    $lastConfirmed = 0 
    $usSummary | Foreach-Object {
        $new = $_.Confirmed - $lastConfirmed
        if($new -lt 0)
        {
            $new = 0
        }
        if($_.Confirmed -gt $lastConfirmed)
        {
            $lastConfirmed = $_.Confirmed
        }
        
        $_ | Add-Member -MemberType NoteProperty -Name new -Value $new
    }

    Write-Verbose "$($usSummary.Count) days" 
    return $usSummary
}

<#
function Get-StateSummary {
    param(
        $StateAbbreviation,
        $StateName,
        $UsAllData,
        [bool] $NoCities = $false
    )

    #Filter out state summaries,
    $waCityFilterScriptBlock = { ($_.'Province/State' -match ".*, $StateAbbreviation" )}
    $waStateFilterScriptBlock = { ($_.'Province/State' -match "${StateName}$" )}
    if($StateName -like '* Princess')
    {
        $waCityFilterScriptBlock = $waStateFilterScriptBlock
    }
    
    return (Get-Summary -UsAllData $usAllData -StateFilterScriptBlock $waStateFilterScriptBlock)
}
#>

$usSummaryTime = Measure-Command {
    $usSummary = Get-Summary -UsAllData $usAllData
}

<#
$stateSummaries = @{}
$statesTime = Measure-Command {
    foreach($state in $stateData)
    {
        Write-Verbose '**********************************' 
        Write-Verbose $state.Name 
        $noCities = $false
        if($state.NoCites)
        {
            $noCities = $true
        }
        $stateSummary = Get-StateSummary -StateAbbreviation $state.abbreviation -StateName $state.Name -UsAllData $usAllData -NoCities:$noCities
        $stateSummaries += @{ "$($state.abbreviation)" =$stateSummary}
        Set-Variable -Name "$($state.abbreviation)Summary" -Value $stateSummary
    }
}
#>

Write-Verbose -Verbose -Message "Summarizing the USA took $($usSummaryTime.TotalSeconds) seconds"
Write-Verbose -Verbose -Message "Last Day : $($usSummary[-1].Date.ToString("yyyy-MM-dd"))"

# Create function to Chart the Summaries

1. A function to Chart the summary for a single area
2. A function to create a stacked chart of multiple areas.

In [ ]:
#Format the traces
function Out-SummaryChart 
{
    param(
        [object] $areaSummary,
        [string] $area
    )
    $usConfirmedTrace = [XPlot.Plotly.Graph+Scatter]::new()

    $usConfirmedTrace.x  = $areaSummary.Date
    $usConfirmedTrace.y  = $areaSummary.Confirmed
    $usConfirmedTrace.text = 'Confirmed'
    $usConfirmedTrace.marker  = $confirmedCasesMarker
    $usConfirmedTrace.name = "$area Confirmed"
    $usConfirmedTrace.line = $line

    $usMortalityTrace = [XPlot.Plotly.Graph+Scatter]::new()

    $usMortalityTrace.x  = $areaSummary.Date
    $usMortalityTrace.y  = $areaSummary.Deaths
    $usMortalityTrace.text = 'Deaths'
    $usMortalityTrace.marker  = $mortalityMarker
    $usMortalityTrace.name = "$area Deaths"


    $usNewTrace = [XPlot.Plotly.Graph+Scatter]::new()

    $usNewTrace.x  = $areaSummary.Date
    $usNewTrace.y  = $areaSummary.New
    $usNewTrace.marker  = $newCasesMarker
    $usNewTrace.name = "$area New Cases"
    $usNewTrace.line = $newCasesLine

    $max = @(($areaSummary.Confirmed | measure-object -max).Maximum)
    $yaxis.dtick = [int] (($max | measure-object -max).Maximum / 5)
    
    # Create the chart
    $layout = [XPlot.Plotly.Layout+Layout]::new()
    $layout.width = 1024
    $layout.height = 500

    $layout.xaxis = $xaxis
    $layout.yaxis = $yaxis
    $layout.title = "John Hopkins - $Area - Coronavirus Data"
    $layout.showlegend = $true
    # Render the chart
    New-PlotlyChart -trace $usConfirmedTrace,$usMortalityTrace, $usNewTrace -Layout $layout  | Out-Display
}

Function Get-HoverInfo 
{
    param(
        [object[]] $data,
        [string] $area
    )
    
    foreach($item in $data)
    {
        "$item - $area"
    }
}

function Out-StackedChart 
{
    param(
        [HashTable] $areaSummary
    )
    $first = true
    $traces = @()
    $max= 

    foreach($area in ($areaSummary.Keys | Sort-Object -Descending))
    {
        $summary = $areaSummary.$area
        
        $usConfirmedTrace = [XPlot.Plotly.Graph+Bar]::new()

        $usConfirmedTrace.x  = $summary.Date
        $usConfirmedTrace.y  = $summary.Confirmed
        $usConfirmedTrace.name = "$area Confirmed"
        $usConfirmedTrace.hoverinfo = Get-HoverInfo -data $Summary.Confirmed -Area $area
        $traces += $usConfirmedTrace
        $max += ($areaSummary.Confirmed | measure-object -max).Maximum
    }

    $yaxis.dtick = [int] (($max | measure-object -max).Maximum / 5)
    
    # Create the chart
    $layout = [XPlot.Plotly.Layout+Layout]::new()
    $layout.width = 1024
    $layout.height = 525

    $layout.xaxis = $xaxis
    $layout.yaxis = $yaxis
    $layout.barmode = 'stack'
    $layout.hovermode = 'closest'
    $layout.title = "John Hopkins - US - Coronavirus Data"
    $layout.showlegend = $false
    # Render the chart
    New-PlotlyChart -trace $traces -Layout $layout  | Out-Display
}

# Use the Summary chart function to graph the summary data for the US

In [ ]:
Out-SummaryChart -AreaSummary $usSummary -area 'US'

# Debugging commands

Please ignore these following cells

In [ ]:
<# Used to verify data
 $usConfirmedByCity | ForEach-Object { $parts = $_.'Province/State' -split ', '; $_ | Add-Member -MemberType NoteProperty -Name State -Value $parts[1] -ErrorAction ignore; $_ | Add-Member -MemberType NoteProperty -Name City -Value $parts[0] -ErrorAction ignore; $_ } | 
    Select-Object -Property city, state, '3/11/20'| group-object -Property state | ForEach-Object { $_.Group 
        | measure-Object -Property '3/11/20' -Sum
            | Add-Member -memberType NoteProperty -Name name -value $_.Name -PassThru } | select-Object -Property Name, Count, Sum
#>

In [ ]:
# Used to verify data
# $usAllData.Confirmed | Where-Object {$_.'Province/State' -like 'Washington'} | Select-Object -Property 'Province/State', '3/14/20' -Unique | Sort-Object -Property 'Province/State'
# $usAllData.Mortality | Where-Object {$_.'Province/State' -like 'Washington'} | Select-Object -Property 'Province/State', '3/14/20' -Unique | Sort-Object -Property 'Province/State'

# list all states
#$usAllData.Mortality | Where-Object {$_.'Province/State' -notlike '*,*'} | Select-Object -Property 'Province/State' -Unique 


In [ ]:
# $usMortalityAll | WHere-Object { $_.'Country/Region' -eq 'US' -and ($_.'Province/State' -notmatch '.*, \w{2}' -or $_.'Province/State' -match '.*Princess' )}